## 🌐 Connect Colab to Google Drive

In [1]:
from google.colab import drive

drive.mount("/gdrive")
%cd /gdrive/My Drive
%cd [2024-2025] AN2DL Homework 2

Mounted at /gdrive
/gdrive/My Drive
/gdrive/My Drive/[2024-2025] AN2DL Homework 2


## ⚙️ Import Libraries

In [ ]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import logging
import random

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from sklearn.model_selection import train_test_split
from scipy.stats import mode

import matplotlib.pyplot as plt
%matplotlib inline

seed = 29
np.random.seed(seed)
tf.random.set_seed(seed)

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

## ⏳ Load the Datasets

In [6]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Training X shape: (2615, 64, 128)
Training y shape: (2615, 64, 128)
Test X shape: (10022, 64, 128)
Input shape: (64, 128, 1)
Number of classes: 5


## 🔍 Inspect the training dataset

In [ ]:
# Extract all the dominant labels
y_train_labels = mode(y_train, axis=(1, 2))[0].flatten()
unique_labels = np.unique(y_train)

# Plot images in batches
def plot_images(X, y, start_index=0, img_row=10, img_col=10):
    fig, axes = plt.subplots(img_col, img_row, figsize=(15, 15))
    for i in range(img_row * img_col):
        idx = start_index + i
        if idx >= len(X):
            break
        ax = axes[i // img_row, i % img_row]
        ax.imshow(X[idx], cmap="gray")
        ax.set_title(f"Class: {y[idx]}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

# Plot just one image from each class
def plot_one(X, y, y_mask, classes):
    for label in classes:
        for i in range(len(y_mask)):
            if label in np.unique(y_mask[i]):
                plt.figure()
                plt.imshow(X[i], cmap="gray")
                plt.title(f"Class: {label}")
                plt.axis("off")
                plt.show()
                break

plot_one(X_train, y_train_labels, y_train, unique_labels)

# Plot all the images in batches
img_row = 10
img_col = 10
img_page = img_row * img_col
num_images = X_train.shape[0]

for start_idx in range(0, num_images, img_page):
    plot_images(X_train, y_train_labels, start_index=start_idx, img_row=img_row, img_col=img_col)

## ❌ Remove outliers from dataset

In [ ]:
# Lists to store the filtered images and labels
X_train_filtered = []
y_train_filtered = []

for i in range(len(y_train)):
    label = y_train[i].argmax() if y_train.ndim > 1 else y_train[i]
    if label != 415:
        # Add to the dataset if not an alien
        X_train_filtered.append(X_train[i])
        y_train_filtered.append(y_train[i])

# Convert filtered lists to NumPy arrays
X_train_filtered = np.array(X_train_filtered)
y_train_filtered = np.array(y_train_filtered)

print(f"Shape X_train_filtered: {X_train_filtered.shape}")
print(f"Shape y_train_filtered: {y_train_filtered.shape}")

num_images_filtered = X_train_filtered.shape[0]

for start_idx in range(0, num_images_filtered, img_page):
    plot_images(X_train_filtered, y_train_filtered, start_index=start_idx, images_per_row=img_row, images_per_col=img_col)

## ✂ Split into Training and Validation Sets

In [12]:
# Split the training dataset to get a validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train_filtered,
    y_train_filtered,
    test_size=0.25,
    random_state=seed)

# Print the shapes of the resulting sets
print('Training set shape:\t', X_train.shape, y_train.shape)
print('Validation set shape:\t', X_val.shape, y_val.shape)

Training set shape:	 (2254, 64, 128, 1) (2254, 64, 128)
Validation set shape:	 (251, 64, 128, 1) (251, 64, 128)


## 🧮 Define network parameters

In [ ]:
# Set batch size for training
batch_size = 64

# Set learning rate for the optimizer
learning_rate = 1e-3

# Set maximum number of training epochs
epochs = 1000

In [ ]:
# Create an EarlyStopping callback
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Create a LearningRate Scheduler, which reduces learning rate if val loss doesn't improve
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=10, min_lr=1e-5
)

# Store the callback in a list
callbacks = [early_stopping, lr_scheduler]

## 🔨 Build the model

In [15]:
def unet_block(input_tensor, filters, kernel_size=3, activation='relu', stack=2, name=''):
    # Initialise the input tensor
    x = input_tensor

    # Apply a sequence of Conv2D, Batch Normalisation, and Activation layers for the specified number of stacks
    for i in range(stack):
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
        x = tfkl.BatchNormalization()(x)
        x = tfkl.Activation(activation)(x)
        x = tfkl.Conv2D(filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
        x = tfkl.BatchNormalization()(x)
        x = tfkl.Activation(activation)(x)
        x = tfkl.SpatialDropout2D(0.2)(x)

    # Return the transformed tensor
    return x

In [ ]:
input_layer = tfkl.Input(shape=input_shape)

# Downsampling path
down_block_1 = unet_block(input_layer, 32)
d1 = tfkl.MaxPooling2D()(down_block_1)

down_block_2 = unet_block(d1, 64)
d2 = tfkl.MaxPooling2D()(down_block_2)

# Bottleneck
bottleneck = unet_block(d2, 128)
x = tfkl.SpatialDropout2D(0.3)(bottleneck)

# Upsampling path
u1 = tfkl.Conv2DTranspose(64, kernel_size=2, strides=2, padding='same')(x)
u1 = tfkl.Concatenate()([u1, down_block_2])
u1 = unet_block(u1, 64)

u2 = tfkl.Conv2DTranspose(64, kernel_size=2, strides=2, padding='same')(u1)
u2 = tfkl.Concatenate()([u2, down_block_1])
u2 = unet_block(u2, 32)

# Output Layer
output_layer = tfkl.Conv2D(num_classes, kernel_size=1, padding='same', activation="softmax")(u2)

model = tfk.Model(inputs=input_layer, outputs=output_layer)

# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False)
optimizer = tfk.optimizers.AdamW(learning_rate=learning_rate, weight_decay=1e-4)
loss = tfk.losses.SparseCategoricalCrossentropy()

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=[mean_iou])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 64, 128, 1)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_21 (Conv2D)        │ (None, 64, 128, 32)    │            320 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_20    │ (None, 64, 128, 32)    │            128 │ conv2d_21[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_20             │ (None, 64, 128, 32)    │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_22 (Conv2D)        │ (None, 64, 128, 32)    │          9,248 │ activation_20[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_21    │ (None, 64, 128, 32)    │            128 │ conv2d_22[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_21             │ (None, 64, 128, 32)    │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spatial_dropout2d_11      │ (None, 64, 128, 32)    │              0 │ activation_21[0][0]    │
│ (SpatialDropout2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_23 (Conv2D)        │ (None, 64, 128, 32)    │          9,248 │ spatial_dropout2d_11[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_22    │ (None, 64, 128, 32)    │            128 │ conv2d_23[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_22             │ (None, 64, 128, 32)    │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_24 (Conv2D)        │ (None, 64, 128, 32)    │          9,248 │ activation_22[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_23    │ (None, 64, 128, 32)    │            128 │ conv2d_24[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_23             │ (None, 64, 128, 32)    │              0 │ batch_normalization_2… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ spatial_dropout2d_12 

 Total params: 968,517 (3.69 MB)

 Trainable params: 965,957 (3.68 MB)

 Non-trainable params: 2,560 (10.00 KB)

## 🛠️ Train and Save the Model

In [18]:
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_data=(X_val, y_val),
    batch_size=batch_size,
    callbacks=callbacks
).history

# Calculate and print the final validation accuracy
final_val_meanIoU = round(max(history['val_mean_io_u_1'])* 100, 2)
print(f'Final validation Mean Intersection Over Union: {final_val_meanIoU}%')

Epoch 1/1000
36/36 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - loss: 1.9536 - mean_io_u_1: 0.0902 - val_loss: 27.0913 - val_mean_io_u_1: 0.0023 - learning_rate: 0.0010
Epoch 2/1000
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - loss: 1.6032 - mean_io_u_1: 0.1652 - val_loss: 36.0696 - val_mean_io_u_1: 0.0015 - learning_rate: 0.0010
Epoch 3/1000
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - loss: 1.4804 - mean_io_u_1: 0.2148 - val_loss: 6.2659 - val_mean_io_u_1: 0.0169 - learning_rate: 0.0010
Epoch 4/1000
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 218ms/step - loss: 1.4171 - mean_io_u_1: 0.2293 - val_loss: 2.6278 - val_mean_io_u_1: 0.0627 - learning_rate: 0.0010
Epoch 5/1000
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 220ms/step - loss: 1.3862 - mean_io_u_1: 0.2305 - val_loss: 3.7095 - val_mean_io_u_1: 0.0758 - learning_rate: 0.0010
Epoch 6/1000
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 223ms/step - loss: 1.3213 - mean_io_u_1: 0.2473 - val_loss: 2.5957 - val_mean_io_u_1: 0.0921 - learning_rate: 0.0010
Epoch 7/1000
36/36 ━━━━━━━━━━━━━━━━━━━

In [ ]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

## 📊 Test the model

In [26]:
# If model_filename is not defined, load the most recent model from Google Drive
if "model_filename" not in globals() or model_filename is None:
    files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith('model_') and f.endswith('.keras')]
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    if files:
        model_filename = files[0]
    else:
        raise FileNotFoundError("No model files found in the current directory.")

model = tfk.models.load_model(model_filename)

preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step
Predictions shape: (10022, 64, 128)


## 💾 Save the predictions

In [27]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [28]:
# Create the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)